In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda'

torch.nn中封装了针对图像处理以及文本处理的多种神经网络处理单元，官方文档：https://pytorch.org/docs/stable/nn.html
其中卷积层convd常见参数包含输入通道，输出通道，卷积核大小，步长等等。
全连接层，在torch中抽象为矩阵乘法，使用`Liner`函数完成，假设输入特征矩阵为`b*n`，`Liner`函数中的参数矩阵为`m*b`，则代表输入特征数量为n， 输出特征数量为m，完成了一次特征降维。
卷积操作特征输入输出大小关系：
$H_{\text {out }}=\left\lfloor\frac{H_{\text {in }}+2 \times \text { padding }[0]-\text { dilation }[0] \times(\text { kernel_size }[0]-1)-1}{\text { stride }[0]}+1\right.$
$W_{\text {out }}=\left\lfloor\frac{W_{\text {in }}+2 \times \operatorname{padding}[1]-\operatorname{dilation}[1] \times(\text { kernel_size }[1]-1)-1}{\text { stride }[1]}+1\right.$
整个神经网络形状为：
第一层：卷积层，输入通道1， 输出通道8，卷积核3*3
第二层：卷积层，输入通道8， 输出通道16， 卷积核3
第三层：全连接层，输入特征为图像大小*通道数量，


In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 卷积层, conv函数位于torch.nn中的图像处理
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.fc1 = nn.Linear(16*6*6, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    
    # 前向计算函数
    def forward(self, x):
        # 卷积层计算
        y = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        y = F.max_pool2d(F.relu(self.conv2(y)), (2,2))
        # 将y形状从2维转换为1维
        y = y.view(x.shape[0], -1)
        y = F.relu(self.fc1(y))
        y = F.relu(self.fc2(y))
        y = F.softmax(self.fc3(y))
        return y
    
net = Net()
print(net)
        
        

Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


检查神经网络参数

In [25]:
params = list(net.parameters())
print(len(params))
print(params[0].shape)

10
torch.Size([8, 1, 3, 3])


生成随机输入，查看输出

In [26]:
testcase = torch.randn(1,1,32,32)
print(testcase)
out = net(testcase)
print(out)

tensor([[[[-0.8900,  0.5631,  1.5987,  ..., -0.6284, -1.5422,  0.9578],
          [-0.0208,  0.3048, -1.6220,  ...,  0.6611, -1.1264,  0.3424],
          [ 0.4162, -0.1956,  0.9835,  ..., -1.1187,  0.1616, -0.4505],
          ...,
          [ 0.2703,  0.2097, -0.2778,  ..., -0.2481,  1.0752,  0.2217],
          [ 1.3621,  0.4516,  1.8471,  ...,  2.2648,  0.4070, -0.8607],
          [-0.6390,  0.5049,  0.2709,  ..., -0.0772,  0.8568,  0.6873]]]])
tensor([[0.0882, 0.1065, 0.0994, 0.1091, 0.1099, 0.0894, 0.0931, 0.1028, 0.0994,
         0.1021]], grad_fn=<SoftmaxBackward>)


c:\users\lihaiyang\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


清零梯度并随机初始化

In [28]:
net.zero_grad()
out.backward(torch.randn(1,10), retain_graph=True)

In [29]:
labels = torch.randn(1,10)
lossF = nn.MSELoss()
loss = lossF(out, labels)
print(loss)

tensor(1.5278, grad_fn=<MseLossBackward>)


In [30]:
net.zero_grad()
print(net.conv1.bias.grad)
loss.backward(retain_graph=True)
print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 6.8219e-04, -9.3590e-04, -6.8058e-05, -3.0262e-04,  1.5181e-03,
        -2.0048e-04,  5.6527e-04,  2.8303e-03])


In [32]:
import torch.optim as optim
print(net.conv1.bias)
optimzer = optim.SGD(net.parameters(), lr=0.5)
optimzer.zero_grad()
out = net(testcase)
loss = lossF(labels, out)
loss.backward()
optimzer.step()
print(net.conv1.bias)

Parameter containing:
tensor([ 0.1990, -0.2244, -0.1236, -0.2516,  0.2725, -0.1386, -0.0248, -0.0824],
       requires_grad=True)
Parameter containing:
tensor([ 0.1989, -0.2241, -0.1235, -0.2512,  0.2722, -0.1386, -0.0251, -0.0833],
       requires_grad=True)


c:\users\lihaiyang\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
